In [1]:
import numpy as np
import serial  # 引用pySerial模組
from time import sleep, time
from emd.sift import sift
from keyboard import is_pressed
import socketio
from collections import deque
# import holoviews as hv
# from holoviews.streams import Buffer
# hv.extension('bokeh')
# renderer = hv.renderer('bokeh')

BAUD_RATE = 115200    # 設定傳輸速率
COM_PORT = 'COM3'    # 指定通訊埠名稱
NUM_IMF = 3
CHANNEL_NUMBER = 3
SAMPLE_RATE = 500
WINDOW_SIZE = 100
NOARDUINO = True
KEY_CLASS = {0:'undefined action', 1:'up', 2:'down', 3:'left', 4:'right', 5:'quick touch'}
MAXCHARLEN = max([len(KEY_CLASS[key]) for key in KEY_CLASS])
REQUEST_CHANNEL = "inferenceRequest"
REQUEST_COOLDOWN = 0.08 #s

record_path = './data/230220_l5m6r7_record_X.npy'
SERVER_URL = 'http://ouo.serveminecraft.net:3000'

In [3]:
class remoteReceiver():
    def __init__(self, url) -> None: 
        self.__clientID = ''
        self.__sio = socketio.Client()
        self.__sio.connect(url)
        self.__sio.on('registerInfo', self.__getID)
        self.__sio.emit('signalHandlerRegister', {'time': "{:.3f}".format(time()), 'remote': True})

        self.__container = deque([], maxlen=WINDOW_SIZE)
        try:
            assert isinstance(self.__sio, socketio.Client)
            print("Connected to Socket server.")
        except:
            raise Exception("Maybe server is not online.")
        
    def __getID(self, id):
        self.__clientID = id
        self.__sio.on(id, self.__getData)
    
    def __getData(self, rcv):
        try:
            self.__container.append(rcv.split(','))
        except:
            pass
        
    def __emdSignal(self, sig):
        sig = np.array(sig).astype(np.float32).reshape(CHANNEL_NUMBER, WINDOW_SIZE).T
        ret = None
    
        for c in range(CHANNEL_NUMBER):
            raw = sig[:, c]
            imf = sift(raw, max_imfs=NUM_IMF, imf_opts={'sd_thresh': 0.1})
            
            if imf.shape[-1] < NUM_IMF:
                compensate = np.zeros((WINDOW_SIZE, NUM_IMF - imf.shape[-1]))
                imf = np.concatenate([imf, compensate], axis = 1)
            
            if not type(ret) == np.ndarray: 
                ret = imf
            else: 
                ret = np.concatenate([ret, imf], axis = 1)
              
        return ",".join(ret[np.newaxis, :].astype(np.str_).flatten().tolist())
        
    def run(self):
        try:
            escape = False
            stamp = time()
            count = 0
            while not escape:
                try:
                    if len(self.__container) == WINDOW_SIZE and time() > (stamp + REQUEST_COOLDOWN):
                        stamp = time()
                        count += 1
                        self.__sio.emit(REQUEST_CHANNEL, {'uid': self.__clientID, 'data': self.__emdSignal(self.__container), 'serial_num': count})
                        print("ID: {} send {: 5d}.".format(self.__clientID, count).ljust(22), end='\r')
                        
                except KeyboardInterrupt:
                    escape = True
                    break
                except:
                    pass
        
        except KeyboardInterrupt:
            self.__sio.disconnect()
            print('Serial disconnected.'.ljust(22))
    
class receiver():
    def __init__(self, url, port, baud_rate, record = None) -> None: 
        if isinstance(record, loader):
            self.__serial = record
        else:
            self.__serial = serial.Serial(port, baud_rate)
            
        self.__clientID = ''
        self.__sio = socketio.Client()
        self.__sio.connect(url)
        self.__sio.on('registerInfo', self.__getID)
        self.__sio.emit('signalHandlerRegister', {'time': "{:.3f}".format(time()), 'remote': False})

        self.__container = deque([], maxlen=WINDOW_SIZE)
        try:
            assert isinstance(self.__sio, socketio.Client)
            print("Connected to Socket server.")
        except:
            raise Exception("Maybe server is not online.")
        
    def __getID(self, id):
        self.__clientID = id
    
    def run0(self): 
        try:
            escape = False
            while not escape:
                while self.__serial.in_waiting:         
                    data_raw = self.__serial.readline() 
                    rcv = data_raw.decode(errors='surrogateescape').rstrip()
                    try:
                        data = np.array(rcv.split(',')).astype(np.float32)
                        if data.shape[0] < CHANNEL_NUMBER:
                            continue
                        self.__sio.emit(REQUEST_CHANNEL, rcv)
                        # rcv_time = round(time() - self.__start, 2)
                    except KeyboardInterrupt:
                        escape = True
                        break
                    except:
                        pass
                    finally:
                        if isinstance(self.__serial, loader):
                            sleep(1 / SAMPLE_RATE)
                            
            raise KeyboardInterrupt

        except KeyboardInterrupt:
            self.__serial.close() 
            self.__sio.disconnect()
            print('Serial disconnected.'.ljust(22))

    def run(self):
        try:
            escape = False
            stamp = time()
            count = 0
            while not escape:
                while self.__serial.in_waiting:
                    data_raw = self.__serial.readline() 
                    rcv = data_raw.decode(errors='surrogateescape').rstrip()
                    try:
                        if len(rcv.split(',')) < CHANNEL_NUMBER:
                            continue
                        self.__container.append(rcv.split(',')) # a0,b0,c0, a1,b1,c1 .... an,bn,cn, <-- tackle this with rstrip() and 
                                                     # reshape with (WINDOW_SIZE, CHANNEL_SIZE) and then transpose -> (CHANNEL_SIZE, WINDOW_SIZE)
                        if len(self.__container) == WINDOW_SIZE and time() > (stamp + REQUEST_COOLDOWN):
                            stamp = time()
                            count += 1
                            self.__sio.emit(REQUEST_CHANNEL, {'uid': self.__clientID, 'data': self.__emdSignal(self.__container), 'serial_num': count})
                            print("ID: {} send {: 5d}.".format(self.__clientID, count).ljust(22), end='\r')
                            
                        # rcv_time = round(time() - self.__start, 2)
                    except KeyboardInterrupt:
                        escape = True
                        break
                    except:
                        pass
                    finally:
                        if isinstance(self.__serial, loader):
                            sleep(1 / SAMPLE_RATE)
                            
            raise KeyboardInterrupt

        except KeyboardInterrupt:
            self.__serial.close() 
            self.__sio.disconnect()
            print('Serial disconnected.'.ljust(22))
            
    def __emdSignal(self, sig):
        sig = np.array(sig).astype(np.float32).reshape(CHANNEL_NUMBER, WINDOW_SIZE).T
        ret = None
    
        for c in range(CHANNEL_NUMBER):
            raw = sig[:, c]
            imf = sift(raw, max_imfs=NUM_IMF, imf_opts={'sd_thresh': 0.1})
            
            if imf.shape[-1] < NUM_IMF:
                compensate = np.zeros((WINDOW_SIZE, NUM_IMF - imf.shape[-1]))
                imf = np.concatenate([imf, compensate], axis = 1)
            
            if not type(ret) == np.ndarray: 
                ret = imf
            else: 
                ret = np.concatenate([ret, imf], axis = 1)
              
        return ",".join(ret[np.newaxis, :].astype(np.str_).flatten().tolist())
            
class recorder():
    def __init__(self, port, baud_rate, record_name = '', record_path = './data', fakeSignal = None) -> None: 
        if isinstance(fakeSignal, loader):
            self.__serial = fakeSignal
        else:
            self.__serial = serial.Serial(port, baud_rate)
        self.X = None
        self.y = []
        if record_name == '':
            self.__record_path = None
        else:
            self.__record_path = record_path + '/' + record_name
    
    def run(self):
        try:
            escape = False
            start = time()
            while not escape:
                while self.__serial.in_waiting:
                    data_raw = self.__serial.readline()  
                    rcv = data_raw.decode(errors='surrogateescape').rstrip() # decode with UTF-8
                    try:
                        #Hand-input labeling
                        label = 0
                        if is_pressed('8'):
                            label = 1
                        elif is_pressed('5'):
                            label = 2
                        elif is_pressed('4'):
                            label = 3
                        elif is_pressed('6'):
                            label = 4
                        elif is_pressed('1'):
                            label = 5
                        elif is_pressed('escape'):
                            raise KeyboardInterrupt

                        data = np.array(rcv.split(',')).astype(np.float32)
                        if data.shape[0] < CHANNEL_NUMBER:
                            continue
                        
                        self.y.append(label)
                        print(("{:.2f}".format(time() - start) + ' ' + KEY_CLASS[self.y[-1]]).ljust(len(KEY_CLASS[0]) + len(str(int(time()))) + 3), end='\r')
                        
                        #Store recieved data
                        if isinstance(self.X, np.ndarray):
                            self.X = np.concatenate([self.X, data[np.newaxis, :].copy()], axis=0)
                        else:
                            self.X = data[np.newaxis, :].copy()              
                            
                    except KeyboardInterrupt:
                        escape = True
                        break
                    except:
                        pass
                            
            raise KeyboardInterrupt

        except KeyboardInterrupt:
            if isinstance(self.__record_path, str):
                #Save recorded data
                np.save(self.__record_path + '_X', self.X)
                print("Saved file {}.".format(self.__record_path + '_X'))

                np.save(self.__record_path + '_y', np.array(self.y).astype(np.int32))
                print("Saved file {}.".format(self.__record_path + '_y'))

            self.__serial.close()
            print('Serial disconnected.'.ljust(50))
            

class loader():
    def __init__(self, path) -> None:
        self.data = np.load(path)
        self.in_waiting = True
        self.__i = 0
        
    def readline(self):
        try:
            ret = ["%.2f" %x for x in self.data[self.__i, :].tolist()]
            return ",".join(ret).encode(encoding='UTF-8')
        except KeyboardInterrupt:
            self.in_waiting = False
        except:
            self.__i = 0
            ret = ["%.2f" %x for x in self.data[self.__i, :].tolist()]
            return ",".join(ret).encode(encoding='UTF-8')
        finally:
            self.__i += 1
    
    def close(self):
        pass

In [ ]:
# (Remote) Recieve real-time signal and send to inference server
signal = remoteReceiver(SERVER_URL)

In [ ]:
# Recieve real-time signal and send to inference server
if NOARDUINO:
    signal = receiver(SERVER_URL, COM_PORT, BAUD_RATE, loader(record_path))
else:
    signal = receiver(SERVER_URL, COM_PORT, BAUD_RATE)

In [4]:
# Recieve, record and label real-time signal
if NOARDUINO:
    signal = recorder(COM_PORT, BAUD_RATE, fakeSignal = loader(record_path))
else:
    record_name = '230220_l5m6r7_record'
    signal = recorder(COM_PORT, BAUD_RATE, record_name)

In [6]:
signal.run()

Saved file ./data/230220_l5m6r7_record_X.
Saved file ./data/230220_l5m6r7_record_y.
Serial disconnected.


In [ ]:
# class RecordPlotter():
#     def __init__(self, record_name = '', record_path = './data/'):
#         self.__buffers = [Buffer(np.zeros((0, 2)), length=int(WINDOW_SIZE//2)) for i in range(CHANNEL_NUMBER)]
#         self.__X = np.load(record_path + record_name + '_X.npy')
#         self.__y = np.load(record_path + record_name + '_y.npy')

#     def buildPlot(self):
#         plt = hv.DynamicMap(hv.Curve, streams=[self.__buffers[0]]).relabel('0')
#         for i in range(1, CHANNEL_NUMBER):
#             plt *= hv.DynamicMap(hv.Curve, streams=[self.__buffers[i]]).relabel(str(i))
#         return plt.opts(padding=0.1, width=600)

#     def streaming(self):
#         escape = False
#         start = time()
#         while not escape:
#             try:
#                 for i in range(self.__X.shape[0]):
#                     for j in range(CHANNEL_NUMBER):
#                         self.__buffers[j].send(np.array([[i, self.__X[i, j]]]))
#                     print(("{:.2f} ".format(time() - start) + KEY_CLASS[self.__y[i]]).ljust(MAXCHARLEN + len(str(int(time()))) + 3), end='\r')
#                     sleep(1 / SAMPLE_RATE)
                    
#             except KeyboardInterrupt:
#                 escape = True
#                 for i in range(CHANNEL_NUMBER):
#                     self.__buffers[i].clear()
#                 break

In [12]:
#Plot record by holoviews in jupyter-notebook
# rp = RecordPlotter('230220_l5m6r7_record')
# rp.buildPlot()

:DynamicMap   []
   :Overlay
      .Curve.A_0 :Curve   [x]   (y)
      .Curve.A_1 :Curve   [x]   (y)
      .Curve.A_2 :Curve   [x]   (y)

In [ ]:
rp.streaming()